## Instalación de dependencias e imports necesarios

In [ ]:
!pip install rarfile

In [ ]:
import os
import rarfile
import cv2
from google.colab import drive

In [ ]:
# ---------------------------------------------
# Paso 1: Montar Google Drive
# ---------------------------------------------
drive.mount('/content/drive')

# Directorio donde están los archivos RAR
rar_train_path = '/content/drive/MyDrive/UPM/AAPI/CBIR/TRAIN.rar'
rar_test_path = '/content/drive/MyDrive/UPM/AAPI/CBIR/TEST.rar'

# Directorio donde se extraerán las imágenes
extract_train_dir = '/content/seg_train'
extract_test_dir = '/content/seg_test'

# Configurar la ruta del ejecutable 'unrar' si es necesario
rarfile.UNRAR_TOOL = '/usr/bin/unrar'

# Extraer el archivo TRAIN.rar
with rarfile.RarFile(rar_train_path) as rar_ref:
    rar_ref.extractall(extract_train_dir)
print(f"Archivos extraídos a {extract_train_dir}")

# Extraer el archivo TEST.rar
with rarfile.RarFile(rar_test_path) as rar_ref:
    rar_ref.extractall(extract_test_dir)
print(f"Archivos extraídos a {extract_test_dir}")

Mounted at /content/drive
Archivos extraídos a /content/seg_train
Archivos extraídos a /content/seg_test


# Pre Procesamiento

## Función para renombrar archivos y actualizar la lista test_images

In [ ]:
def rename_images_in_directory(base_dir):
    renamed_images = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                # Obtén el nombre de la carpeta
                folder_name = os.path.basename(root)
                # Crea el nuevo nombre del archivo
                new_name = f"{folder_name}_{file}"
                # Construye rutas completas
                old_path = os.path.join(root, file)
                new_path = os.path.join(root, new_name)
                # Renombra el archivo
                os.rename(old_path, new_path)
                # Agrega el nuevo nombre a la lista
                renamed_images.append(new_path)
    return renamed_images

# Renombrar archivos en el directorio de test
test_images = rename_images_in_directory(extract_test_dir)

# Imprimir resultados
print(f"Total de imágenes de test: {len(test_images)}")
print(f"Ejemplo de imágenes renombradas: {test_images[:10]}")

Total de imágenes de test: 120
Ejemplo de imágenes renombradas: ['/content/seg_test/TEST/Edificios/Edificios_22227.jpg', '/content/seg_test/TEST/Edificios/Edificios_20601.jpg', '/content/seg_test/TEST/Edificios/Edificios_24103.jpg', '/content/seg_test/TEST/Edificios/Edificios_20206.jpg', '/content/seg_test/TEST/Edificios/Edificios_23106.jpg', '/content/seg_test/TEST/Edificios/Edificios_22969.jpg', '/content/seg_test/TEST/Edificios/Edificios_24058.jpg', '/content/seg_test/TEST/Edificios/Edificios_22150.jpg', '/content/seg_test/TEST/Edificios/Edificios_22234.jpg', '/content/seg_test/TEST/Edificios/Edificios_20245.jpg']


## Redimensionamiento

In [ ]:
def resize_image(image, size=(150, 150)):
    """
    Redimensiona una imagen al tamaño especificado, manejando correctamente ampliaciones y reducciones.

    Args:
        image: La imagen a redimensionar (numpy array).
        size: Una tupla (alto, ancho) con el tamaño deseado.

    Returns:
        La imagen redimensionada.
    """
    # Obtenemos las dimensiones actuales de la imagen
    current_height, current_width = image.shape[:2]

    # Si las dimensiones no coinciden con el tamaño deseado, redimensionamos
    if current_height != size[0] or current_width != size[1]:
        # Calcular el factor de escala para redimensionar proporcionalmente
        scale_height = size[0] / current_height
        scale_width = size[1] / current_width

        # Elegimos el menor factor de escala para que se ajuste dentro del tamaño
        scale = min(scale_height, scale_width)

        # Nuevas dimensiones escaladas proporcionalmente
        new_width = int(current_width * scale)
        new_height = int(current_height * scale)

        # Elegir el método de interpolación adecuado
        if scale > 1:  # Si se está ampliando la imagen
            interpolation = cv2.INTER_CUBIC
        else:  # Si se está reduciendo la imagen
            interpolation = cv2.INTER_AREA

        # Redimensionamos al tamaño escalado
        resized_image = cv2.resize(image, (new_width, new_height), interpolation = interpolation)

        # Creamos un lienzo de fondo negro con el tamaño final
        final_image = np.zeros((size[0], size[1], 3), dtype=np.uint8)

        # Centramos la imagen redimensionada en el lienzo
        y_offset = (size[0] - new_height) // 2
        x_offset = (size[1] - new_width) // 2
        final_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

        return final_image
    else:
        # Si ya tiene el tamaño deseado, devolver la imagen original
        return image

## Reducción de ruido

### Reducción de ruido para Handcrafted

In [ ]:
def reducir_ruido_handcrafted(image):
    """
    Reducción de ruido específica para imágenes utilizadas en el método Handcrafted.
    Preserva bordes importantes mientras elimina ruido.

    Args:
        image: Imagen cargada con OpenCV (escala de grises o color).

    Returns:
        Imagen con reducción de ruido.
    """
    # Convertir la imagen a escala de grises (si no lo está ya)
    if len(image.shape) == 3:  # Si la imagen tiene 3 canales (BGR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro bilateral para preservar bordes
    denoised_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

    # Convertir la imagen a color (RGB) si está en escala de grises
    if len(denoised_image.shape) == 2:  # Si la imagen tiene un solo canal (grayscale)
        denoised_image = cv2.cvtColor(denoised_image, cv2.COLOR_GRAY2BGR)

    return denoised_image

# Cambios resto del codigo


## VGG16 (agregacion de redimensionamiento)

In [ ]:
# ....................



    # Aplicar la función resize_image para redimensionar (224x224 es requerido por VGG16)
    resized_img = resize_image(img, size=(224, 224))

    # Convertir de BGR (OpenCV) a RGB (Keras) y procesar como array
    img_array = image.img_to_array(resized_img)

# ......................................

## Handcrafted (agregacion de redimensionamiento, filtro de ruido, y semilla en la caracteristica extraccion_color_dcd)

In [ ]:
class Handcrafted:
    def __init__(self, imagen, target_size=(150, 150)):
        imagen = cv2.imread(imagen)
        if imagen is None:
            raise ValueError(f"No se pudo leer la imagen {imagen}")
        # Reducción de ruido
        imagen = reducir_ruido_handcrafted(imagen)
        self.imagen_color = resize_image(imagen, size = target_size)
        self.imagen_grayscale = cv2.cvtColor(self.imagen_color, cv2.COLOR_BGR2GRAY)

        # Inicialización de vectores de características
        self.vector_color = []
        self.vector_textura = []
        self.vector_forma = []
        self.vector_hog = []
        self.vector_histograma_color = []
        self.vector_sobel = []
        self.vector_fft = []
        self.vector_hu = []
        self.vector_gabor = []

    def extraccion_color_dcd(self, num_colores_dominantes=8, random_seed=42):
        self.vector_color = np.zeros(3 * num_colores_dominantes, dtype=float)
        pixels = self.imagen_color.reshape(-1, 3)
        kmeans = MiniBatchKMeans(n_clusters=num_colores_dominantes, n_init=10, batch_size=10000,
        random_state=random_seed) # Semilla fijada
        kmeans.fit(pixels)
        colores_dominantes = kmeans.cluster_centers_
        self.vector_color[:] = colores_dominantes.flatten()
        print(f"Vector Color DCD: {len(self.vector_color)}")

#................. todo lo demas igual

# Evaluación del modelo con métricas globales

## Función de evaluación

In [ ]:
def evaluar_modelo(img_queries, n_imgs, feature_extractor):
    import timeit
    precisiones = []
    recalls = []
    tiempos = []

    for img_query in img_queries:
        start_time = timeit.default_timer()

        # Recuperar imágenes similares
        ranking, image_paths = retrieve_image_with_scores(img_query, feature_extractor, n_imgs)

        # Calcular tiempo
        end_time = timeit.default_timer()
        tiempos.append(end_time - start_time)

        # Evaluar precisión y recall
        precision_k, recall_k = calcular_precision_recall(img_query, ranking, n_imgs, image_paths)
        precisiones.append(precision_k)
        recalls.append(recall_k)

    # Métricas finales
    precision_media = sum(precisiones) / len(precisiones)
    recall_medio = sum(recalls) / len(recalls)
    tiempo_medio = sum(tiempos) / len(tiempos)

    print(f"Precisión Media: {precision_media * 100:.2f}%")
    print(f"Recall Medio: {recall_medio * 100:.2f}%")
    print(f"Tiempo Medio: {tiempo_medio:.2f}s")

    return precision_media, recall_medio, tiempo_medio

# Probar el modelo
img_queries = test_images[:120]  # 10 imágenes para prueba
n_imgs = 10  # Número de imágenes similares a recuperar
feature_extractor = 'Extractor 2'  # Cambia a 'Extractor 2' para usar VGG16

# Evaluar y mostrar resultados
vgg16_precision_media, vgg16_recall_medio, vgg16_tiempo_medio = evaluar_modelo(img_queries, n_imgs, feature_extractor)
feature_extractor = 'Extractor 1'  # Cambia a 'Extractor 2' para usar VGG16
handcrafted_precision_media, handcrafted_recall_medio, handcrafted_tiempo_medio = evaluar_modelo(img_queries, n_imgs, feature_extractor)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 942ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 908ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 961ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 984ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 570ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 994ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 958ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

## Resultados de métricas globales

In [ ]:
print("Handcrafted")
print(f"Precisión Media: {handcrafted_precision_media * 100:.2f}%")
print(f"Recall Medio: {handcrafted_recall_medio * 100:.2f}%")
print(f"Tiempo Medio: {handcrafted_tiempo_medio:.2f}s")

print("VGG16")
print(f"Precisión Media: {vgg16_precision_media * 100:.2f}%")
print(f"Recall Medio: {vgg16_recall_medio * 100:.2f}%")
print(f"Tiempo Medio: {vgg16_tiempo_medio:.2f}s")

Handcrafted
Precisión Media: 32.50%
Recall Medio: 10.83%
Tiempo Medio: 0.79s
VGG16
Precisión Media: 88.67%
Recall Medio: 29.56%
Tiempo Medio: 0.86s
